In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
# df_artists = pd.read_csv(r'/home/agastya/Documents/Python/Praxis/ML/Personal Practice/Spotify_Project/data/data_by_artist.csv', ',')

In [4]:
df_genres = pd.read_csv(r'/content/drive/My Drive/Spotify Project/data_by_genres.csv', ',')

In [5]:
df_w_genres = pd.read_csv(r'/content/drive/My Drive/Spotify Project/data_w_genres.csv', ',')

In [6]:
# Drop Unnamed column
df_w_genres.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
# Convert genres from string to list 
import re
pattern = re.compile(r'\w+\s*?\w+', re.IGNORECASE)
df_w_genres['genres'] = df_w_genres['genres'].map(lambda x: re.findall(pattern, x))

In [8]:
# Change fields of no genres to None
df_w_genres['genres'] = df_w_genres['genres'].map(lambda x: np.nan if len(x) == 0 else x)

In [9]:
# Artists with no genres
no_genre_artists = df_w_genres[df_w_genres['genres'].isna()]

In [ ]:
no_genre_artists

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres
72,Orchestra Studio 7,0.342315,0.561378,203097.777311,0.392441,0.467496,0.190490,-14.332546,0.054264,116.684315,0.532022,0.021008,5,1,238,NaN
88,Sinclair Lewis,0.691179,0.681108,264736.075472,0.208925,0.000005,0.258333,-19.368524,0.955585,106.152203,0.460193,0.004717,4,1,212,NaN
90,Unspecified,0.812928,0.462446,141540.142180,0.344851,0.377778,0.283078,-18.818299,0.154392,110.679294,0.474366,1.241706,10,1,211,NaN
124,Orchestra Sinfonica di Roma della RAI,0.968614,0.379123,204830.590643,0.193932,0.046241,0.268602,-16.807421,0.103258,103.132047,0.212032,0.000000,10,1,171,NaN
130,Rita Ampatzi,0.953272,0.459373,195420.538462,0.448793,0.331829,0.193202,-9.905633,0.050520,111.843538,0.725680,0.011834,1,1,169,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27589,Rock City,0.096300,0.749000,239973.000000,0.846000,0.012200,0.313000,-7.518000,0.089500,143.004000,0.481000,50.000000,1,1,1,NaN
27592,DMC,0.188000,0.458000,326733.000000,0.824000,0.000000,0.112000,-4.224000,0.519000,86.331000,0.499000,43.000000,11,0,1,NaN
27595,Suzi Benson,0.495000,0.552000,247493.000000,0.391000,0.000000,0.086700,-12.097000,0.034300,131.980000,0.309000,44.000000,2,1,1,NaN
27596,Sam Burtis,0.152000,0.662000,374813.000000,0.764000,0.010300,0.080300,-7.994000,0.081900,135.902000,0.769000,31.000000,7,1,1,NaN


# Distance Calculation #


In [14]:
# For next code cells
def standardize_tempo_for_genre(x):
    min_tempo = df_genres['tempo'].min()
    max_tempo = df_genres['tempo'].max()
    return (x-min_tempo)/(max_tempo-min_tempo)

In [15]:
def standardize_tempo_for_artist(x):
    min_tempo = no_genre_artists['tempo'].min()
    max_tempo = no_genre_artists['tempo'].max()
    return (x-min_tempo)/(max_tempo-min_tempo)

In [ ]:
# Try 1 - [acousticness, danceability, instrumentalness, tempo, valence]

try_1_genres = df_genres.loc[:, ['acousticness', 'danceability', 'instrumentalness', 'tempo', 'valence']]
try_1_genres['tempo'] = try_1_genres['tempo'].map(standardize_tempo_for_genre)

try_1_artists = no_genre_artists.loc[:, ['acousticness', 'danceability', 'instrumentalness', 'tempo', 'valence']]
try_1_artists['tempo'] = try_1_artists['tempo'].map(standardize_tempo_for_artist)

try_1_artists = np.array(try_1_artists)
try_1_genres = np.array(try_1_genres) 

In [ ]:
artist_genre_distance_try_1 = []
artist_genre_indexes_try_1 = []

for artist_feature in try_1_artists:
    distance_vals = []
    for genre_feature in try_1_genres:
        distance_vals.append(np.linalg.norm(artist_feature-genre_feature))
    # Take only top 10
    distance_vals = np.array(distance_vals)
    
    artist_genre_distance_try_1.append(distance_vals[distance_vals.argsort()][:10])
    artist_genre_indexes_try_1.append(distance_vals.argsort()[:10])

In [ ]:
no_genre_artists.iloc[0]

artists             Orchestra Studio 7
acousticness                  0.342315
danceability                  0.561378
duration_ms                     203098
energy                        0.392441
instrumentalness              0.467496
liveness                       0.19049
loudness                      -14.3325
speechiness                  0.0542639
tempo                          116.684
valence                       0.532022
popularity                   0.0210084
key                                  5
mode                                 1
count                              238
genres                             NaN
Name: 72, dtype: object

In [ ]:
df_genres.iloc[artist_genre_indexes_try_1[0]]

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
2057,rhode island indie,0.248761,0.505667,188386.166667,0.728417,0.439298,0.148358,-8.770500,0.058208,132.339750,0.581667,41.083333,4,1
331,british industrial,0.260124,0.596156,253703.606803,0.632908,0.501976,0.157053,-13.414348,0.061914,121.039166,0.560386,29.080612,0,1
231,belgian indie,0.467000,0.564250,286366.500000,0.688750,0.486250,0.100200,-8.171500,0.036225,129.002000,0.506000,46.500000,10,1
259,bleep techno,0.243382,0.572361,273238.888889,0.736333,0.507456,0.208069,-8.836778,0.069403,130.764778,0.614436,43.555556,11,1
1605,modern dream pop,0.408820,0.535250,250775.749036,0.596949,0.386814,0.137330,-8.953715,0.037197,124.788873,0.462639,49.222039,2,1
1240,instrumental soul,0.381695,0.602435,252408.701452,0.539435,0.518661,0.135659,-11.004841,0.047014,115.386640,0.607844,32.983241,10,1
1865,palm desert scene,0.203318,0.482522,232229.779894,0.737249,0.427242,0.192502,-8.504152,0.056501,126.326448,0.528291,44.356878,9,1
797,downtempo,0.266034,0.603996,297572.058136,0.579686,0.424080,0.166288,-9.493431,0.066456,118.680109,0.429592,49.203524,7,1
2206,smooth jazz,0.337245,0.632402,293887.727558,0.539757,0.417892,0.139156,-10.970289,0.040750,112.243773,0.583466,37.752913,5,1
123,atmospheric post-rock,0.252210,0.668000,331466.500000,0.773500,0.478055,0.144000,-5.724000,0.056000,119.999500,0.483000,23.500000,11,1


In [ ]:
# Try 2 - [acousticness, danceability, instrumentalness, tempo,energy, valence, energy]

try_2_genres = df_genres.loc[:, ['acousticness', 'danceability', 'instrumentalness', 'energy', 'tempo', 'valence']]
try_2_genres['tempo'] = try_2_genres['tempo'].map(standardize_tempo_for_genre)

try_2_artists = no_genre_artists.loc[:, ['acousticness', 'danceability', 'instrumentalness', 'energy', 'tempo', 'valence']]
try_2_artists['tempo'] = try_2_artists['tempo'].map(standardize_tempo_for_artist)

try_2_artists = np.array(try_2_artists)
try_2_genres = np.array(try_2_genres) 

In [ ]:
artist_genre_distance_try_2 = []
artist_genre_indexes_try_2 = []

for artist_feature in try_2_artists:
    distance_vals = []
    for genre_feature in try_2_genres:
        distance_vals.append(np.linalg.norm(artist_feature-genre_feature))
    # Take only top 10
    distance_vals = np.array(distance_vals)
    
    artist_genre_distance_try_2.append(distance_vals[distance_vals.argsort()][:10])
    artist_genre_indexes_try_2.append(distance_vals.argsort()[:10])

In [ ]:
df_genres.iloc[artist_genre_indexes_try_1[4]]

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
2039,rebetiko,0.959770,0.533861,193814.327377,0.376418,0.336413,0.193219,-11.768717,0.053218,114.612810,0.716307,2.008617,7,1
2535,vintage old-time,0.968875,0.578806,184259.888889,0.267389,0.304343,0.192260,-12.567111,0.069199,122.886611,0.737694,4.125000,10,1
1247,irish banjo,0.837333,0.483000,172618.000000,0.601333,0.248893,0.693333,-11.168333,0.167533,128.974333,0.773667,25.666667,2,1
2541,vintage tango,0.945190,0.564508,177501.059780,0.310706,0.356746,0.243507,-11.680452,0.101448,116.664835,0.643147,4.227987,2,1
1844,orquesta tipica,0.924137,0.607385,163619.052656,0.321117,0.404575,0.291078,-11.650894,0.122958,133.062939,0.700014,4.205128,9,1
418,carnatic vocal,0.907121,0.550103,318522.668240,0.388231,0.253515,0.214114,-11.422840,0.091707,113.549147,0.680970,9.106392,10,1
1079,greek clarinet,0.844585,0.476566,171165.840931,0.522081,0.293887,0.231821,-9.599911,0.067806,110.036666,0.723124,0.000000,2,1
2534,vintage jazz,0.928986,0.610030,183929.619837,0.317587,0.329331,0.225722,-12.599809,0.076543,120.875787,0.654530,6.689927,5,1
1084,greek swing,0.978595,0.571524,191273.166667,0.336893,0.254362,0.230271,-13.069369,0.051024,113.645512,0.683095,0.595238,7,1
517,classic bollywood,0.885505,0.502630,230690.737137,0.354426,0.308211,0.213069,-11.773005,0.097147,109.769526,0.633942,8.191785,7,1


In [ ]:
df_w_genres.head(1)

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres
0,Francisco Canaro,0.983072,0.654711,177776.513454,0.292622,0.490675,0.201118,-11.733373,0.111007,123.608786,0.746469,0.054257,2,1,2267,"[tango, vintage tango]"


2602

In [ ]:
df_w_genres[~df_w_genres['genres'].isna()].shape

(18056, 16)

# Multilabel Classification #

In [16]:
# For sparse df
genre_set = set()
for i in df_w_genres['genres']:
    # print(type(i))
    try:
        genre_set.update(i)
    except(TypeError):
        pass

genre_set = list(sorted(genre_set)) # Number of unique genres

In [33]:
'''# Use pickle file instead
genre_sparse_df = pd.DataFrame(np.zeros((18056, len(genre_set))), columns=genre_set)
index = 0
for i in df_w_genres[~df_w_genres['genres'].isna()]['genres']:
    for j in i:
        genre_sparse_df.loc[index, j] = 1
    index +=1'''

In [17]:
from scipy.sparse import csr_matrix, csc_matrix

In [36]:
'''from scipy.sparse import csr_matrix, csc_matrix
genre_sparse_csr = csr_matrix(genre_sparse_df)
# To save the sparse matrix
with open('csr_matrix_sorted', 'wb') as f:
    pickle.dump(genre_sparse_csr, f)'''

In [ ]:
# To save the full sparse dataset
# genre_sparse_df.to_pickle('sparse_matrix')

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
training_raw = df_w_genres[~df_w_genres['genres'].isna()]

In [20]:
training_raw = training_raw.loc[:, ['acousticness', 'danceability', 'instrumentalness', 'energy', 'tempo', 'valence']]

In [21]:
training_raw['tempo'] = training_raw['tempo'].map(standardize_tempo_for_artist)

In [22]:
with open('/content/drive/My Drive/Spotify Project/csr_matrix_sorted', 'rb') as f:
    genre_sparse_csr = pickle.load(f)

In [39]:
genre_sparse_csr.toarray().sum(axis=0)

array([ 2.,  2., 11., ...,  2.,  1.,  8.])

In [41]:
df_w_genres['genres']

0                                   [tango, vintage tango]
1        [classical, early romantic, era, polish classi...
2          [classical, classical era, early romantic, era]
3                               [classical, classical era]
4        [baroque, classical, early music, german baroque]
                               ...                        
27601                                       [memphis soul]
27602    [big room, edm, electro house, progressive hou...
27603                              [south carolina, indie]
27604    [alternative dance, brighton indie, dance, pun...
27605                                    [australian rock]
Name: genres, Length: 27606, dtype: object

In [23]:
# Use toarray to convert it back to a full set
genre_sparse_csr.toarray().shape

(18056, 2602)

In [26]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [27]:
dt = MultiOutputClassifier(DecisionTreeClassifier(max_depth=4), n_jobs=-1)
dt.fit(training_raw, genre_sparse_csr.toarray())

MultiOutputClassifier(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=4,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       presort='deprecated',
                                                       random_state=None,
                

In [28]:
test_set = no_genre_artists.loc[:, ['acousticness', 'danceability', 'instrumentalness', 'energy', 'tempo', 'valence']]

In [29]:
genre_labels = dt.predict(test_set)

In [30]:
genre_labels.sum(axis=0).mean() # On average, 27 genres for each artist!!

27.017294388931592

In [31]:
genre_labels_results_df = pd.DataFrame(genre_labels, columns=genre_set)

In [43]:
genre_labels_results_df.head()

,21st century,432hz,a cappella,abstract,abstract beats,abstract hip,abstract idm,accordeon,accordion,acid house,acid jazz,acid rock,acid trance,acousmatic,acoustic,acoustic blues,acoustic pop,acoustic punk,adoracion,adult standards,adventista,afghan pop,afghan traditional,african gospel,african percussion,african reggae,african rock,afro,afro dancehall,afro house,afro psych,afrobeat,afrobeat brasileiro,afrofuturism,afrofuturismo brasileiro,afropop,aggrotech,ahead jazz,alabama indie,alabama metal,...,wight indie,wind ensemble,wind quintet,wisconsin indie,witch house,women,wonky,woogie,wop,world,world fusion,world meditation,world worship,worship,wrestling,wu fam,wyoming indie,wyoming roots,xhosa,yacht rock,ye ye,yeladim,yiddish folk,yo,yodeling,yoga,yoik,york indie,yugoslav new,yugoslav rock,zapstep,zen,zhongguo feng,zimdancehall,zither,zolo,zouglou,zouk,zouk riddim,zydeco
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
stacked_labels = genre_labels_results_df.stack().reset_index()

In [70]:
# stacked_labels[stacked_labels[0]==1].groupby('level_0')['level_1'].agg(lambda x: ','.join(x).split(', '))

level_0
0       [brass band,classic mandopop,dixieland,doo,mai...
1       [chicano rap,classic mandopop,dixieland,doo,ma...
2       [brass band,classic mandopop,dixieland,doo,mai...
3                                      [doo,mandopop,wop]
4              [brass band,dixieland,doo,new orleans,wop]
                              ...                        
9545    [brass band,classic mandopop,dark trap,dixiela...
9546    [brass band,classic mandopop,dixieland,doo,hip...
9547    [brass band,classic mandopop,dixieland,doo,mai...
9548    [brass band,classic mandopop,dixieland,doo,mai...
9549    [dixieland,doo,jug band,new orleans,samoan pop...
Name: level_1, Length: 9548, dtype: object

In [72]:
labels = stacked_labels[stacked_labels[0]==1].groupby('level_0').agg(lambda x:[i for i in x])['level_1']

In [73]:
test_set['genres'] = labels

In [85]:
df_w_genres.iloc[130]

artists             Rita Ampatzi
acousticness            0.953272
danceability            0.459373
duration_ms               195421
energy                  0.448793
instrumentalness        0.331829
liveness                0.193202
loudness                -9.90563
speechiness            0.0505201
tempo                    111.844
valence                  0.72568
popularity             0.0118343
key                            1
mode                           1
count                        169
genres                       NaN
Name: 130, dtype: object

In [84]:
test_set.head(5)

,acousticness,danceability,instrumentalness,energy,tempo,valence,genres
72,0.342315,0.561378,0.467496,0.392441,116.684315,0.532022,"[doo, mandopop, wop]"
88,0.691179,0.681108,0.000005,0.208925,106.152203,0.460193,"[brass band, classic mandopop, dixieland, doo,..."
90,0.812928,0.462446,0.377778,0.344851,110.679294,0.474366,"[brass band, chicano rap, dixieland, doo, new ..."
124,0.968614,0.379123,0.046241,0.193932,103.132047,0.212032,"[doo, mandopop, wop]"
130,0.953272,0.459373,0.331829,0.448793,111.843538,0.725680,"[dixieland, doo, mandopop, new orleans, wop]"


In [79]:
df_genres[df_genres['genres'].map(lambda x: 'mandopop' in x)]

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
538,classic mandopop,0.563406,0.536419,276468.988744,0.414222,0.001727,0.243067,-11.293855,0.036354,109.406931,0.346907,42.743750,11,1
1502,malaysian mandopop,0.590625,0.485792,265370.833333,0.445292,0.000000,0.150283,-8.148375,0.031338,111.477500,0.245125,55.125000,7,1
1512,mandopop,0.512112,0.552523,271718.351458,0.460823,0.000269,0.171492,-8.654063,0.032484,120.681434,0.349007,53.051800,7,1
2176,singaporean mandopop,0.614200,0.564961,261460.094444,0.465750,0.000003,0.170792,-8.113228,0.032812,123.040783,0.335111,55.316667,6,1
